<a href="https://colab.research.google.com/github/carlibeisel/mason_water_budget/blob/main/reach_compile_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Compile data for Mason Water Budget**

By: Carli Beisel

Created: August 2024

**Variables:**

*qin* = streamflow at upstream gauge on reach

*qout* = streamflow at downstream gauge on reach

*sr* = surface runoff from agricultural irrigation

*gw* = groundwater

*pet* = potential evapotranspiration

PET is estiamted using this PET for open water in Kuna: https://et-idwr.idaho.gov/stcvrstats.py?station=1068&cover=55&stats=Potential

## 1. Import Libraries and Datasets ##

In [ ]:
## --------------- ##
## IMPORT PACKAGES ##
## --------------- ##

import pandas as pd # to work with dataframe
import os # for file paths
import glob # read in a folder of csv
import numpy as np # basic statistics

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## --------------- ##
##  LOAD DATASETS  ##
## --------------- ##

drainflow = pd.read_csv('')

sr = pd.read_csv('')

pet = pd.read_csv('/content/drive/MyDrive/Data/Mason/data_output/pet_out/pet_estimate.csv') # rough estiamtes from IDWR (mm/day)
#pet = pd.read_csv('/content/drive/MyDrive/Data/Mason/data_output/pet_out/pet_reach.csv')

gw = pd.read_csv('')

precip = 0 # rough estimate . . .
#precip = pd.read_csv('/content/drive/MyDrive/Data/Mason/data_output/precip_out/precip_reach.csv')

NameError: name 'qin' is not defined

In [ ]:
## --------------------------- ##
##  Convert rates to volumes   ##
## --------------------------- ##

## 2. Merge Datasets ##

In [ ]:
## ------------------------------------------ ##
## Use a dictionary to help align reach names ##
## ------------------------------------------ ##

In [ ]:
## ------------------------------ ##
##   Merge into one .csv file     ##
## ------------------------------ ##

output_directory = '/content/drive/MyDrive/Data/Mason/compiled_data/reach/'

In [ ]:
## ------------------------ ##
##   Export merged file     ##
## ------------------------ ##

file = pd.read_csv('/content/drive/MyDrive/Data/Mason/compiled_data/merged_water_budget.csv')